In [1]:
# Env setup
import folium  # http://python-visualization.github.io/folium/docs-v0.5.0/quickstart.html
from folium import plugins
import geojson
import sqlite3
import pandas as pd
import os
from os import path

In [ ]:
# Connect to database
connect = sqlite3.connect(path.abspath(path.join('..', '..', 'sample_files', 'db', 'week5_assignment_branch_atm.sqlite')))
cursor = connect.cursor()

In [3]:
# Retrieve data
branch_data_raw = pd.read_sql('''
    SELECT 
        a.street,
        a.town,
        a.zip_code,
        b.title as branch_type,
        a.email,
        a.phone,
        a.latitude,
        a.longitude,
        ('Monday: ' || a.oh_monday ||
            '; Tuesday:' || a.oh_tuesday || 
                '; Wednesday: ' || a.oh_wednesday ||
                    '; Thursday: ' || a.oh_thursday ||
                        '; Friday: ' || a.oh_friday ||
                            '; Saturday: ' || a.oh_saturday ||
                                '; Sunday: ' || a.oh_sunday) as opening_hours,
        a.services,
        a.consultants
    FROM branch_atm a
    LEFT JOIN branch_type_cl b on a.branch_type_id = b.id
    ORDER BY b.id ASC;
    ''', connect)

# Data preview
branch_data_raw.head()

,street,town,zip_code,branch_type,email,phone,latitude,longitude,opening_hours,services,consultants
0,Galantská cesta 20,Dunajská Streda,92901,Bank branch,info@csob.sk,0850 111 777,47.99606,17.629435,Monday: 10:00 - 18:00; Tuesday:10:00 - 18:00; ...,"Open at weekends and during holidays, Wheelcha...",
1,Laurinská 1,Bratislava,81101,Bank branch,info@csob.sk,0850 111 777,48.14300,17.109169,Monday: 08:00 - 12:00 - 13:00 - 17:00; Tuesday...,"Safe deposit box, Wheelchair access",
2,Nám. SNP 29,Bratislava,81101,Bank branch,info@csob.sk,0850 111 777,48.14520,17.110891,Monday: 09:00 - 18:00; Tuesday:09:00 - 18:00; ...,"Safe deposit box, Private banking, Wheelchair ...",
3,Nám. SNP 41,Zvolen,96001,Bank branch,info@csob.sk,0850 111 777,48.57750,19.125140,Monday: 08:30 - 16:30; Tuesday:08:30 - 16:30; ...,"Wheelchair access, Parking",
4,Toryská 5,Košice,04011,Bank branch,info@csob.sk,0850 111 777,48.71420,21.240124,Monday: 09:00 - 20:00; Tuesday:09:00 - 20:00; ...,"Open at weekends and during holidays, Wheelcha...",


In [4]:
# Number of different branch types
counts_branch_types = branch_data_raw['branch_type'].value_counts()
print(counts_branch_types)

ATM                             265
Bank branch                     110
Insurance branch                 68
Building savings bank branch     45
Leasing branch                   12
Name: branch_type, dtype: int64


In [5]:
# Convert pandas df to GeoJSON
def data2geojson(df):
    features = list()
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["longitude"],
                                                    X["latitude"],
                                                    0)),  # no data for elevation
                            properties=dict(name=X["town"],
                                            description=X["street"])))
    df.apply(insert_features, axis=1)
    return geojson.dumps(geojson.FeatureCollection(features), sort_keys=True, ensure_ascii=False)

In [6]:
# Folium demo examples http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples
# Awesome icons https://fontawesome.com/v4.7.0/icons/
# Base map
csob_map = folium.Map(
    location = [48.6737532, 19.696058],  # centre of Slovakia
    zoom_start = 8,
    tiles = 'CartoDBPositron')

folium.TileLayer('CartoDBPositron').add_to(csob_map)
folium.TileLayer('OpenStreetMap').add_to(csob_map)

# Feature groups
# Bank branches
bank_group = folium.FeatureGroup(name='Bank branches', overlay = True, show = True)
# Markers
for row in range(0, len(branch_data_raw.loc[branch_data_raw['branch_type'] == 'Bank branch'])):
    folium.Marker(
        location = [branch_data_raw.iloc[row]['latitude'], branch_data_raw.iloc[row]['longitude']],
        popup = 
        '<b>Type: </b>' + branch_data_raw.iloc[row]['branch_type'] + '<br />' +
        '<b>Street: </b>' + branch_data_raw.iloc[row]['street'] + '<br />' +
        '<b>Town: </b>' + branch_data_raw.iloc[row]['town'] + '<br />' +
        '<b>Zip code: </b>' + branch_data_raw.iloc[row]['zip_code'] + '<br />' +
        '<b>Opening hours: </b>' + branch_data_raw.iloc[row]['opening_hours'] + '<br />',
        icon = folium.Icon(color = 'blue', icon = 'bank', prefix = 'fa')
    ).add_to(bank_group)
    
# Feature groups
# ATMs
atm_group = folium.FeatureGroup(name='ATMs', overlay = True, show = False)
# Markers
for row in range(0, len(branch_data_raw.loc[branch_data_raw['branch_type'] == 'ATM'])):
    folium.Marker(
        location = [branch_data_raw.iloc[row]['latitude'], branch_data_raw.iloc[row]['longitude']],
        popup = 
        '<b>Type: </b>' + branch_data_raw.iloc[row]['branch_type'] + '<br />' +
        '<b>Street: </b>' + branch_data_raw.iloc[row]['street'] + '<br />' +
        '<b>Town: </b>' + branch_data_raw.iloc[row]['town'] + '<br />' +
        '<b>Zip code: </b>' + branch_data_raw.iloc[row]['zip_code'] + '<br />' +
        '<b>Opening hours: </b>' + branch_data_raw.iloc[row]['opening_hours'] + '<br />',
        icon = folium.Icon(color = 'red', icon = 'money', prefix = 'fa')
    ).add_to(atm_group)
    
# Feature groups
# Insurance branches
insurance_group = folium.FeatureGroup(name='Insurance branches', overlay = True, show = False)
# Markers
for row in range(0, len(branch_data_raw.loc[branch_data_raw['branch_type'] == 'Insurance branch'])):
    folium.Marker(
        location = [branch_data_raw.iloc[row]['latitude'], branch_data_raw.iloc[row]['longitude']],
        popup = 
        '<b>Type: </b>' + branch_data_raw.iloc[row]['branch_type'] + '<br />' +
        '<b>Street: </b>' + branch_data_raw.iloc[row]['street'] + '<br />' +
        '<b>Town: </b>' + branch_data_raw.iloc[row]['town'] + '<br />' +
        '<b>Zip code: </b>' + branch_data_raw.iloc[row]['zip_code'] + '<br />' +
        '<b>Opening hours: </b>' + branch_data_raw.iloc[row]['opening_hours'] + '<br />',
        icon = folium.Icon(color = 'green', icon = 'heartbeat', prefix = 'fa')
    ).add_to(insurance_group)
    
# Feature groups
# Building savings bank branches
bsb_group = folium.FeatureGroup(name='Building savings bank branches', overlay = True, show = False)
# Markers
for row in range(0, len(branch_data_raw.loc[branch_data_raw['branch_type'] == 'Building savings bank branch'])):
    folium.Marker(
        location = [branch_data_raw.iloc[row]['latitude'], branch_data_raw.iloc[row]['longitude']],
        popup = 
        '<b>Type: </b>' + branch_data_raw.iloc[row]['branch_type'] + '<br />' +
        '<b>Street: </b>' + branch_data_raw.iloc[row]['street'] + '<br />' +
        '<b>Town: </b>' + branch_data_raw.iloc[row]['town'] + '<br />' +
        '<b>Zip code: </b>' + branch_data_raw.iloc[row]['zip_code'] + '<br />' +
        '<b>Opening hours: </b>' + branch_data_raw.iloc[row]['opening_hours'] + '<br />',
        icon = folium.Icon(color = 'orange', icon = 'home', prefix = 'fa')
    ).add_to(bsb_group)

# Feature groups
# Leasing branches
leasing_group = folium.FeatureGroup(name='Leasing branches', overlay = True, show = False)
# Markers
for row in range(0, len(branch_data_raw.loc[branch_data_raw['branch_type'] == 'Leasing branch'])):
    folium.Marker(
        location = [branch_data_raw.iloc[row]['latitude'], branch_data_raw.iloc[row]['longitude']],
        popup = 
        '<b>Type: </b>' + branch_data_raw.iloc[row]['branch_type'] + '<br />' +
        '<b>Street: </b>' + branch_data_raw.iloc[row]['street'] + '<br />' +
        '<b>Town: </b>' + branch_data_raw.iloc[row]['town'] + '<br />' +
        '<b>Zip code: </b>' + branch_data_raw.iloc[row]['zip_code'] + '<br />' +
        '<b>Opening hours: </b>' + branch_data_raw.iloc[row]['opening_hours'] + '<br />',
        icon = folium.Icon(color = 'gray', icon = 'car', prefix = 'fa')
    ).add_to(leasing_group)

    
# Layer control
bank_group.add_to(csob_map)
atm_group.add_to(csob_map)
insurance_group.add_to(csob_map)
bsb_group.add_to(csob_map)
leasing_group.add_to(csob_map)
folium.LayerControl().add_to(csob_map)


# Search plugin
# plugins.Search(data2geojson(branch_data_raw), position='topright').add_to(csob_map)    

In [7]:
# Print map
csob_map

In [8]:
# Save map
csob_map.save('csob_branch_atm_map.html')